In [3]:
%matplotlib inline

In [4]:
import sys
import numpy as np
import pandas as pd
import re as re
import matplotlib.pyplot as plt
import xgboost as xgb

In [3]:
train_uid = pd.read_table('../data/train/uid_train.txt', header = None, names = ['uid', 'label'])

In [4]:
print(train_uid.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 2 columns):
uid      4999 non-null object
label    4999 non-null int64
dtypes: int64(1), object(1)
memory usage: 78.2+ KB
None


In [51]:
train_voice = pd.read_table('../data/train/voice_train.txt', header = None, names = ['uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'end_time', 'call_type', 'in_out'], dtype = {'opp_num':str})

In [52]:
print(train_voice.head())

     uid                           opp_num opp_head  opp_len  start_time  \
0  u0113  38D54642A237A11BB18455FC1E505292      132       11    26115956   
1  u0113  38D54642A237A11BB18455FC1E505292      132       11    26115623   
2  u0113  38D54642A237A11BB18455FC1E505292      132       11    26174233   
3  u0113  38D54642A237A11BB18455FC1E505292      132       11    26070423   
4  u3340  010A66F2AD42C48C44897A3DEC96A2A1      139       11    26201745   

   end_time  call_type  in_out  
0  26120033          1       1  
1  26115707          1       1  
2  26174321          1       1  
3  26070512          1       0  
4  26201825          1       1  


In [7]:
train_sms = pd.read_table('../data/train/sms_train.txt', header = None, names = ['uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'in_out'], dtype = {'opp_num':str})

In [8]:
print(train_sms.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302976 entries, 0 to 302975
Data columns (total 6 columns):
uid           302976 non-null object
opp_num       302976 non-null object
opp_head      302976 non-null int64
opp_len       302976 non-null int64
start_time    302976 non-null int64
in_out        302976 non-null int64
dtypes: int64(4), object(2)
memory usage: 13.9+ MB
None


In [9]:
train_wa = pd.read_table('../data/train/wa_train.txt', header = None, names = ['uid', 'wa_name', 'visit_cnt', 'visit_dura', 'up_flow', 'down_flow', 'wa_type', 'date'])

In [10]:
print(train_wa.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4808343 entries, 0 to 4808342
Data columns (total 8 columns):
uid           object
wa_name       object
visit_cnt     float64
visit_dura    float64
up_flow       float64
down_flow     float64
wa_type       float64
date          float64
dtypes: float64(6), object(2)
memory usage: 293.5+ MB
None


In [11]:
train_wa

,uid,wa_name,visit_cnt,visit_dura,up_flow,down_flow,wa_type,date
0,u0001,155导航,5.0,207.0,313.0,457.0,0.0,14.0
1,u0001,155导航,7.0,396.0,547.0,659.0,0.0,4.0
2,u0001,155导航,10.0,3212.0,781.0,941.0,0.0,12.0
3,u0001,155导航,14.0,723.0,1094.0,1318.0,0.0,10.0
4,u0001,155导航,18.0,990.0,1406.0,1694.0,0.0,11.0
5,u0001,360借条,2.0,105.0,112.0,272.0,1.0,12.0
6,u0001,360借条,4.0,7428.0,3680.0,1076.0,1.0,9.0
7,u0001,360借条,10.0,11419.0,10535.0,3434.0,1.0,1.0
8,u0001,360借条,14.0,10051.0,13635.0,4318.0,1.0,11.0
9,u0001,360借条,22.0,17542.0,29674.0,8774.0,1.0,4.0


In [17]:
train_wa_df = train_wa[True^train_wa['wa_name'].isnull()]

In [21]:
train_wa_df

,uid,wa_name,visit_cnt,visit_dura,up_flow,down_flow,wa_type,date
0,u0001,155导航,5.0,207.0,313.0,457.0,0.0,14
1,u0001,155导航,7.0,396.0,547.0,659.0,0.0,4
2,u0001,155导航,10.0,3212.0,781.0,941.0,0.0,12
3,u0001,155导航,14.0,723.0,1094.0,1318.0,0.0,10
4,u0001,155导航,18.0,990.0,1406.0,1694.0,0.0,11
5,u0001,360借条,2.0,105.0,112.0,272.0,1.0,12
6,u0001,360借条,4.0,7428.0,3680.0,1076.0,1.0,9
7,u0001,360借条,10.0,11419.0,10535.0,3434.0,1.0,1
8,u0001,360借条,14.0,10051.0,13635.0,4318.0,1.0,11
9,u0001,360借条,22.0,17542.0,29674.0,8774.0,1.0,4


In [24]:
train_wa_df.loc[:,'date'] = train_wa_df.loc[:,'date'].astype(int)

In [26]:
print(train_wa_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4807542 entries, 0 to 4808342
Data columns (total 8 columns):
uid           object
wa_name       object
visit_cnt     float64
visit_dura    float64
up_flow       float64
down_flow     float64
wa_type       float64
date          int64
dtypes: float64(5), int64(1), object(2)
memory usage: 330.1+ MB
None


In [28]:
train_wa_df.loc[:,'visit_cnt'] = train_wa_df.loc[:,'visit_cnt'].astype(int)

In [29]:
print(train_wa_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4807542 entries, 0 to 4808342
Data columns (total 8 columns):
uid           object
wa_name       object
visit_cnt     int64
visit_dura    float64
up_flow       float64
down_flow     float64
wa_type       float64
date          int64
dtypes: float64(4), int64(2), object(2)
memory usage: 330.1+ MB
None


In [30]:
train_wa_df.loc[:,['up_flow', 'down_flow']] = train_wa_df.loc[:,['up_flow','down_flow']].astype(int)

In [31]:
print(train_wa_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4807542 entries, 0 to 4808342
Data columns (total 8 columns):
uid           object
wa_name       object
visit_cnt     int64
visit_dura    float64
up_flow       int64
down_flow     int64
wa_type       float64
date          int64
dtypes: float64(2), int64(4), object(2)
memory usage: 330.1+ MB
None


In [32]:
train_wa_df.loc[:,['visit_dura', 'wa_type']] = train_wa_df.loc[:,['visit_dura', 'wa_type']].astype(int)

In [33]:
print(train_wa_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4807542 entries, 0 to 4808342
Data columns (total 8 columns):
uid           object
wa_name       object
visit_cnt     int64
visit_dura    int64
up_flow       int64
down_flow     int64
wa_type       int64
date          int64
dtypes: int64(6), object(2)
memory usage: 330.1+ MB
None


In [34]:
train_wa_df.loc[:,'wa_name'] = train_wa_df.loc[:,'wa_name'].astype(str)

In [36]:
print(train_wa_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4807542 entries, 0 to 4808342
Data columns (total 8 columns):
uid           object
wa_name       object
visit_cnt     int64
visit_dura    int64
up_flow       int64
down_flow     int64
wa_type       int64
date          int64
dtypes: int64(6), object(2)
memory usage: 330.1+ MB
None


In [41]:
print(train_wa_df.head(10))

     uid wa_name  visit_cnt  visit_dura  up_flow  down_flow  wa_type  date
0  u0001   155导航          5         207      313        457        0    14
1  u0001   155导航          7         396      547        659        0     4
2  u0001   155导航         10        3212      781        941        0    12
3  u0001   155导航         14         723     1094       1318        0    10
4  u0001   155导航         18         990     1406       1694        0    11
5  u0001   360借条          2         105      112        272        1    12
6  u0001   360借条          4        7428     3680       1076        1     9
7  u0001   360借条         10       11419    10535       3434        1     1
8  u0001   360借条         14       10051    13635       4318        1    11
9  u0001   360借条         22       17542    29674       8774        1     4


In [42]:
train_wa_df.to_csv('../data/train_wa_df.csv', index=False)

In [62]:
train_voice_df = train_voice

In [63]:
train_voice_df.loc[:,'in_out'] = train_voice_df.loc[:,'in_out'].astype(int)

In [64]:
print(train_voice_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150778 entries, 0 to 1150777
Data columns (total 8 columns):
uid           1150778 non-null object
opp_num       1150778 non-null object
opp_head      1150778 non-null object
opp_len       1150778 non-null int64
start_time    1150778 non-null int64
end_time      1150778 non-null int64
call_type     1150778 non-null int64
in_out        1150778 non-null int64
dtypes: int64(5), object(3)
memory usage: 70.2+ MB
None


In [65]:
train_voice_df.to_csv('../data/train_voice_df.csv', index=False)

In [67]:
print(train_sms.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302976 entries, 0 to 302975
Data columns (total 6 columns):
uid           302976 non-null object
opp_num       302976 non-null object
opp_head      302976 non-null int64
opp_len       302976 non-null int64
start_time    302976 non-null int64
in_out        302976 non-null int64
dtypes: int64(4), object(2)
memory usage: 13.9+ MB
None


In [69]:
train_sms_df = train_sms

In [70]:
train_sms_df.to_csv('../data/train/train_sms_df.csv', index=False)

In [19]:
def do2(x):
    x = int(x.replace('u', ''))
    return x
def do(dataset):
    ans = dataset
    ans.loc[:, 'uid'] = ans['uid'].apply(do2)
    
    return ans

In [20]:
train_uid_df = pd.read_csv('../data/train/train_uid_df.csv', header = 0)
tmp = train_uid_df
train_uid_df_2 = do(tmp)

In [21]:
print(train_uid_df_2)

       uid  label
0        1      0
1        2      0
2        3      0
3        4      0
4        5      0
5        6      0
6        7      0
7        8      0
8        9      0
9       10      0
10      11      0
11      12      0
12      13      0
13      14      0
14      15      0
15      16      0
16      17      0
17      18      0
18      19      0
19      20      0
20      21      0
21      22      0
22      23      0
23      24      0
24      25      0
25      26      0
26      27      0
27      28      0
28      29      0
29      30      0
...    ...    ...
4969  4970      1
4970  4971      1
4971  4972      1
4972  4973      1
4973  4974      1
4974  4975      1
4975  4976      1
4976  4977      1
4977  4978      1
4978  4979      1
4979  4980      1
4980  4981      1
4981  4982      1
4982  4983      1
4983  4984      1
4984  4985      1
4985  4986      1
4986  4987      1
4987  4988      1
4988  4989      1
4989  4990      1
4990  4991      1
4991  4992      1
4992  4993

In [22]:
train_voice_df = pd.read_csv('../data/train/train_voice_df.csv', header = 0, dtype = {'opp_num':str})
train_sms_df = pd.read_csv('../data/train/train_sms_df.csv', header = 0, dtype = {'opp_num':str})
train_wa_df = pd.read_csv('../data/train/train_wa_df.csv', header = 0)

d:\program files (x86)\python\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
train_voice_df_2 = do(train_voice_df)

In [25]:
print(train_voice_df)

          uid                           opp_num opp_head  opp_len  start_time  \
0         113  38D54642A237A11BB18455FC1E505292      132       11    26115956   
1         113  38D54642A237A11BB18455FC1E505292      132       11    26115623   
2         113  38D54642A237A11BB18455FC1E505292      132       11    26174233   
3         113  38D54642A237A11BB18455FC1E505292      132       11    26070423   
4        3340  010A66F2AD42C48C44897A3DEC96A2A1      139       11    26201745   
5        3340  BC16EA96F7447486FE25AC2BB7A665EF      185       11    26060320   
6        3340  EE497EAB9DA2FB0805390237F983D454      100        7    26061046   
7        3340  010A66F2AD42C48C44897A3DEC96A2A1      139       11    26192655   
8        2016  5D2654B214731E788C5206A96BD6DA59      139       11    26112314   
9        2016  084501372689ECABD58FBB3E7FF254B8      131       11    26112217   
10       2016  5D2654B214731E788C5206A96BD6DA59      139       11    26094342   
11       2016  5D2654B214731

In [26]:
train_sms_df_2 = do(train_sms_df)
train_wa_df_2 = do(train_wa_df)

In [27]:
print(train_sms_df_2)

         uid                           opp_num  opp_head  opp_len  start_time  \
0       4003  B378E065731B897E7295926B27CBA0D5       186       11    20174042   
1       4003  B378E065731B897E7295926B27CBA0D5       186       11    20174130   
2       4003  1B15607F3E6D167B44D46046D5993D87       189       11    20015746   
3       4003  1B15607F3E6D167B44D46046D5993D87       189       11    20015640   
4       4003  B378E065731B897E7295926B27CBA0D5       186       11    20045300   
5       4003  B378E065731B897E7295926B27CBA0D5       186       11    20045253   
6       2354  FFCB99E153F74A7A6BB2481E9EAA666D       132       11    20104305   
7       2354  FFCB99E153F74A7A6BB2481E9EAA666D       132       11    20104305   
8       2354  FFCB99E153F74A7A6BB2481E9EAA666D       132       11    20170619   
9       2354  7FEFA80914C7C9336EEC31BAEC427E14       106        9    20090930   
10      2354  7FEFA80914C7C9336EEC31BAEC427E14       106        9    20090930   
11      2354  FFCB99E153F74A

In [28]:
print(train_wa_df_2)

          uid    wa_name  visit_cnt  visit_dura  up_flow  down_flow  wa_type  \
0           1      155导航          5         207      313        457        0   
1           1      155导航          7         396      547        659        0   
2           1      155导航         10        3212      781        941        0   
3           1      155导航         14         723     1094       1318        0   
4           1      155导航         18         990     1406       1694        0   
5           1      360借条          2         105      112        272        1   
6           1      360借条          4        7428     3680       1076        1   
7           1      360借条         10       11419    10535       3434        1   
8           1      360借条         14       10051    13635       4318        1   
9           1      360借条         22       17542    29674       8774        1   
10          1      360借条         24       16316    19319       6324        1   
11          1   360安全中心网          3     

In [29]:
train_uid_df_2.to_csv('../data/train/train_uid_df.csv', index=False)
train_voice_df_2.to_csv('../data/train/train_voice_df.csv', index=False)
train_sms_df_2.to_csv('../data/train/train_sms_df.csv', index=False)
train_wa_df_2.to_csv('../data/train/train_wa_df.csv', index=False)